The discriminant of $f(x) = \prod_{i=1}^m (x - \alpha_i)$ is $\Delta(f) = \prod_{i<j} (\alpha_i - \alpha_j)^2$. The discriminant is zero if and only if at least two roots are equal, i.e. $f$ has a repeated root.

If a polynomial $f(x)$ has a repeated root $\alpha$, then $\alpha$ is a root of both $f(x)$ and its derivative $f'(x)$. Recall that the resultant of two polynomials $\mathop{Res}(f, g)$ is zero if and only if they have a common root. By choosing $g(x) = f'(x)$, we see that $\mathop{Res}(f, f') = 0$ if and only if $f$ has a repeated root. This shows that the discriminant and the resultant must be proportional to each other.

We have the formula for the resultant
\begin{equation}
    \mathop{Res}(f, f') = a_m^{m-1} \prod_{i=1}^m f'(\alpha_i).
\end{equation}
The value of the derivative at a root $\alpha_k$ is
\begin{equation}
    f'(\alpha_k) = a_m \prod_{j \neq k} (\alpha_k - \alpha_j).
\end{equation}
Now, substitute this back into the resultant formula to obtain
\begin{equation}
    \mathop{Res}(f, f') = a_m^{m-1} a_m^m \prod_{k=1}^m \prod_{j \neq k} (\alpha_k - \alpha_j).
\end{equation}
The double product term contains every pair $(\alpha_k - \alpha_j)$ where $k \neq j$, hence we can group these terms
\begin{equation}
    \prod_k \prod_{j \neq k} (\alpha_k - \alpha_j) = \prod_{k<j} -(\alpha_k - \alpha_j)^2 = (-1)^{m(m-1)/2} \prod_{k<j} (\alpha_k - \alpha_j)^2.
\end{equation}
The final part of this expression is precisely the definition of the discriminant $\Delta(f)$. In the end, we have
\begin{equation}
    \Delta(f) = \frac{(-1)^{m(m-1)/2}}{a_m} \mathop{Res}(f, f')
\end{equation}
where $m$ is the degree of the polynomial $f$ and $a_m$ is the leading coefficient of $f$. For a monic polynomial the formula simplifies to
\begin{equation}
    \Delta(f) = (-1)^{m(m-1)/2} \mathop{Res}(f, f').
\end{equation}



---

We shall try to compute the resultant of two polynomials in $x$ whose coefficients are themselves polynomials in $y$, i.e., they are elements of $Z[y][x]$. The resultant will be a polynomial in $y$. Solving for the roots of this resultant polynomial gives the $y$-coordinates of the points where the two original equations intersect. We use an evaluation and interpolation method, taking advantage of the fact that a polynomial of degree $r$ is uniquely determined by any $r+1$ value.

*   Establish a degree bound: The resultant of two polynomials $f(x,y)$ and $g(x,y)$ (with respect to $x$) is a polynomial in $y$. A bound on the degree of this resultant polynomial can be calculated as
\begin{equation}
    r = \deg_x(f) \deg_y(g) + \deg_x(g) \deg_y(f).
\end{equation}
This bound tells us that the resultant polynomial is uniquely determined by $r + 1$ points.
*   Evaluation: The program chooses $r + 1$ distinct integer values for $y$, for instance, $y = 0, \dots, r$. For each value $y_i$, it substitutes this value into $f(x,y)$ and $g(x,y)$, producing two new polynomials $f_i(x)$ and $g_i(x)$ with integer coefficients.
*   Pointwise resultant calculation: For each pair of integer-coefficient polynomials $f_i(x)$ and $g_i(x)$, we compute their integer resultant $R_i$. This yields $r + 1$ points $(y_i, R_i)$ that lie on the final resultant polynomial.
*   Interpolation: Using the $r + 1$ points, we can reconstruct the resultant polynomial $\mathop{Res}(y)$. We do this via Newton's method of interpolation which is suitable for integer arithmetic.

As an application to solve the systems of bivariate polynomials, the integer roots of the computed resultant polynomial $\mathop{Res}(y)$ are found. For each valid $y$ root, the corresponding $x$ value is found by substituting $y$ back into the original equations and finding the common root of the resulting single-variable polynomials in $x$.

In [7]:
import numpy as np

class Polynomial:
    def __init__(self, coeffs):
        self.coeffs = self._strip_zeros([int(c) for c in coeffs])

    @staticmethod
    def _strip_zeros(coeffs):
        if not coeffs or all(c == 0 for c in coeffs): return [0]
        first_nonzero = next(i for i, c in enumerate(coeffs) if c != 0)
        return coeffs[first_nonzero:]

    @property
    def degree(self):
        if len(self.coeffs) == 1 and self.coeffs[0] == 0: return -1
        return len(self.coeffs) - 1

    def is_zero(self): return self.degree == -1

    def evaluate(self, x): return np.polyval(self.coeffs, x)

    def __add__(self, other):
        max_len = max(len(self.coeffs), len(other.coeffs))
        c1 = [0]*(max_len - len(self.coeffs)) + self.coeffs
        c2 = [0]*(max_len - len(other.coeffs)) + other.coeffs
        return Polynomial([a + b for a, b in zip(c1, c2)])

    def __mul__(self, other):
        return Polynomial(np.convolve(self.coeffs, other.coeffs).tolist())

    def __str__(self):
        if self.is_zero(): return "0"
        parts = []
        for i, c in enumerate(self.coeffs):
            p = self.degree - i
            if c == 0: continue
            cs = str(c); vs = ""
            if c == 1 and p != 0: cs = ""
            if c == -1 and p != 0: cs = "-"
            if p > 1: vs = f"y^{p}"
            elif p == 1: vs = "y"
            parts.append(f"{cs}{vs}")
        return " + ".join(parts).replace(" + -", " - ")

def resultant_integer(f_coeffs, g_coeffs):
    f, g = Polynomial(f_coeffs), Polynomial(g_coeffs)
    m, n = f.degree, g.degree
    if n == 0: return f.evaluate(0) if g.is_zero() else g.coeffs[0] ** m if m >= 0 else 1
    if m < n: return ((-1)**(m*n)) * resultant_integer(g.coeffs, f.coeffs)
    # Using a simplified PRS based on the property Res(f,g) = (-1)^mn Res(g, f mod g) for fields
    rem_coeffs = np.polydiv(np.array(f.coeffs, dtype=float) * (g.coeffs[0]**(m-n+1)), np.array(g.coeffs, dtype=float))[1]
    rem_coeffs = [round(c) for c in rem_coeffs]
    rem = Polynomial(rem_coeffs)
    if rem.is_zero(): return 0
    factor = g.coeffs[0]**(m - rem.degree)
    if (m*n) % 2 != 0: factor *= -1
    return factor * resultant_integer(g.coeffs, rem.coeffs) // (g.coeffs[0]**(m-n+1))**(n-rem.degree)

In [8]:
class BivariatePolynomial:
    def __init__(self, coeffs_as_lists_in_y):
        self.coeffs = [Polynomial(c) for c in coeffs_as_lists_in_y]

    @property
    def degree_x(self): return len(self.coeffs) - 1

    @property
    def degree_y(self): return max(c.degree for c in self.coeffs if not c.is_zero())

    def evaluate_y(self, y_val):
        return Polynomial([c.evaluate(y_val) for c in self.coeffs])

def resultant_bivariate_interp(f, g):
    m, n, m_y, n_y = f.degree_x, g.degree_x, f.degree_y, g.degree_y
    degree_bound = m * n_y + n * m_y

    points = []
    for i in range(degree_bound + 1):
        y_val = i
        f_at_y, g_at_y = f.evaluate_y(y_val), g.evaluate_y(y_val)
        res_val = resultant_integer(f_at_y.coeffs, g_at_y.coeffs)
        points.append((y_val, res_val))

    return newton_interpolation(points)

def newton_interpolation(points):
    n = len(points)
    x = [p[0] for p in points]; y = [p[1] for p in points]
    F = [[0]*n for _ in range(n)]
    for i in range(n): F[i][0] = y[i]
    for j in range(1, n):
        for i in range(j, n):
            F[i][j] = (F[i][j-1] - F[i-1][j-1]) // (x[i] - x[i-j])

    newton_coeffs = [F[k][k] for k in range(n)]
    current_poly = Polynomial([0]); basis_poly = Polynomial([1])
    for k in range(n):
        term = Polynomial([newton_coeffs[k]]) * basis_poly
        current_poly = current_poly + term
        basis_poly = basis_poly * Polynomial([1, -x[k]])
    return current_poly

def find_integer_roots(p):
    roots = []
    if p.coeffs[-1] == 0: roots.append(0)
    divisors = [i for i in range(1, abs(p.coeffs[-1]) + 1) if p.coeffs[-1] % i == 0]
    for d in divisors:
        if p.evaluate(d) == 0: roots.append(d)
        if p.evaluate(-d) == 0: roots.append(-d)
    return sorted(list(set(roots)))

def poly_gcd(f_coeffs, g_coeffs):
    '''
    Computes the GCD of two polynomials using the Euclidean algorithm.
    '''
    # Use numpy's poly1d objects for easier polynomial arithmetic
    f = np.poly1d(f_coeffs)
    g = np.poly1d(g_coeffs)

    while not np.allclose(g.coeffs, 0):
        # The core of the Euclidean algorithm: f = q*g + r
        q, r = np.polydiv(f, g)
        f, g = g, r

    # Normalize the GCD to have a leading coefficient of 1
    if f.order >= 0 and not np.isclose(f.coeffs[0], 0):
        f /= f.coeffs[0]

    return f.coeffs

def solve_system(f, g):
    # Calculating the resultant
    res_y = resultant_bivariate_interp(f, g)
    print(f"Resultant Res_x(f, g) = {res_y}")

    y_sols = find_integer_roots(res_y)
    print(f"Possible y-values for solutions: {y_sols}")

    solutions = set()
    for y in y_sols:
        # Substitute y to get polynomials in x
        f_x_poly = f.evaluate_y(y)
        g_x_poly = g.evaluate_y(y)

        # Compute the GCD to find common roots
        gcd_coeffs = poly_gcd(f_x_poly.coeffs, g_x_poly.coeffs)

        # The roots of the GCD are the common x-solutions
        if len(gcd_coeffs) > 0 and not (len(gcd_coeffs) == 1 and gcd_coeffs[0] == 0):
            x_sols = np.roots(gcd_coeffs)

            for x in x_sols:
                # Add real solutions, cleaning up floating point noise
                if abs(x.imag) < 1e-9:
                    solutions.add((round(x.real, 5), y))

    print(f"Solutions (x, y): {sorted(list(solutions))}")


f1 = BivariatePolynomial([[2], [-2, 6], [-3, 1, 4]])
g1 = BivariatePolynomial([[3], [-3], [-2, -6, -4]])
print("Solving system 1")
solve_system(f1, g1)

f2 = BivariatePolynomial([[2], [3, -1], [2, -2, -4]])
g2 = BivariatePolynomial([[5], [4, 0], [4, 0, -16]])
print("\nSolving system 2")
solve_system(f2, g2)

Solving system 1
Resultant Res_x(f, g) = 3y^4 - 123y^2 + 216y + 336
Possible y-values for solutions: [-7, -1, 4]
Solutions (x, y): [(np.float64(-4.0), 4), (np.float64(0.0), -1), (np.float64(5.0), -7), (np.float64(5.0), 4)]

Solving system 2
Resultant Res_x(f, g) = 160y^4 - 160y^3 - 480y^2 + 160y + 320
Possible y-values for solutions: [-1, 1, 2]
Solutions (x, y): [(np.float64(-2.0), 1), (np.float64(0.0), 2), (np.float64(2.0), -1)]
